In [8]:
import numpy as np
import pandas as pd
import matplotlib as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [11]:
# Demo data file from Eric (example of output from IAA)
iaa = pd.read_csv("IAA-file-demo.csv")
iaa

,article_num,article_sha256,article_id,article_filename,source_task_uuid,tua_uuid,namespace,schema_sha256,question_Number,answer_uuid,question_type,agreed_Answer,coding_perc_agreement,highlighted_indices,agreement_score,num_users,num_answer_choices,target_text,question_text,answer_text,answer_content
0,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,270ea60c-f961-402b-96d5-c015fac1960d,a23b77bd-9eae-4050-9b5d-e38ecb126226,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,1,73d7a14a-9ec6-404c-b2b7-a55508af3b76,checklist,1,1.000000,"[3764, 3765, 3766, 3767, 3768, 3769, 3770, 377...",1.000000,3,3,"EspeciallyÂ at risk are the elderly, who both ...",Is a general or singular causal claim made? Hi...,"General Causation (In general, X causes Y.)",NaN
1,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,270ea60c-f961-402b-96d5-c015fac1960d,a23b77bd-9eae-4050-9b5d-e38ecb126226,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,1,0,checklist,L,0.000000,[],0.000000,3,3,L,Is a general or singular causal claim made? Hi...,L,NaN
2,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,270ea60c-f961-402b-96d5-c015fac1960d,a23b77bd-9eae-4050-9b5d-e38ecb126226,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,1,0,checklist,L,0.000000,[],0.000000,3,3,L,Is a general or singular causal claim made? Hi...,L,NaN
3,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,270ea60c-f961-402b-96d5-c015fac1960d,a23b77bd-9eae-4050-9b5d-e38ecb126226,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,0,checklist,L,0.000000,[],0.000000,3,9,L,What evidence is given for the primary causal ...,L,NaN
4,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,270ea60c-f961-402b-96d5-c015fac1960d,a23b77bd-9eae-4050-9b5d-e38ecb126226,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,0,checklist,L,0.000000,[],0.000000,3,9,L,What evidence is given for the primary causal ...,L,NaN
5,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,270ea60c-f961-402b-96d5-c015fac1960d,a23b77bd-9eae-4050-9b5d-e38ecb126226,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,0,checklist,L,0.000000,[],0.000000,3,9,L,What evidence is given for the primary causal ...,L,NaN
6,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,270ea60c-f961-402b-96d5-c015fac1960d,a23b77bd-9eae-4050-9b5d-e38ecb126226,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,0546a258-029e-46ed-a84f-9ea36d69a770,checklist,4,0.666667,"[3764, 3765, 3766, 3767, 3768, 3769, 3770, 377...",0.666667,3,9,"EspeciallyÂ at risk are the elderly, who both ...",What evidence is given for the primary causal ...,A plausible mechanism is proposed,NaN
7,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b537e0ed21179a29ed28da28057d338e67330ae...,Covid_article_for_PE_S&S&S.txt,270ea60c-f961-402b-96d5-c015fac1960d,a23b77bd-9eae-4050-9b5d-e38ecb126226,Covid_Evidence2020_03_21,45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...,2,780989ca-2765-4989-9cc3-08f3874489e0,checklist,5,0.666667,"[4043, 4044, 4045, 4046, 4047, 4048, 4049, 405...",0.666667,3,9,"In February, a study from the National Academi...",What evidence is given for the primary causal ...,An experimental study was conducted (natural e...,NaN
8,100059,4b537e0ed21179a29ed28da28057d338e67330ae12123c...,raw_304b5

In [15]:
iaa["tua_uuid"][3]

'a23b77bd-9eae-4050-9b5d-e38ecb126226'

Important columns:
- article_num (maybe not needed)
- article_sha256 (part of unique answer identifier)
- tua_uuid (part of unique answer identifier)
- schema_namespace (human readable schema type)
- schema_sha256 (part of unique answer identifier)
- num_answer_choices (helpful for scoring ordinal questions)
- question_Number (part of unique answer identifier)
- answer_uuid (what we want to find)
- agreed_answer (is a number if )
- question_type (checklist, nominal, ordinal)

Function that takes in **article_sha256** (string), **tua_uuid** (string), **schema_sha256** (string), and **question_Number** (int), and returns the **corresponding consensus answer** from IAA.

The return value is a single int instead of the "T1.Q1.A1" format because I believe IAA processes and outputs just the number. 

(also let me know if there's a more efficient way to index the df)

In [49]:
def answer_int_to_string(question, answer):
    return "T1.Q" + str(question) + ".A" + str(answer)

In [50]:
def get_consensus(article_sha256, tua_uuid, schema_sha256, question_Number):
    answer_df = iaa.loc[(iaa["article_sha256"] == article_sha256)
                         & (iaa["tua_uuid"] == tua_uuid)
                         & (iaa["schema_sha256"] == schema_sha256)
                         & (iaa["question_Number"] == question_Number)]
    
    question_type = answer_df["question_type"].iloc[0]
    
    if (question_type == "checklist"):
        answer_list = list(answer_df["agreed_Answer"].values)
        return [answer_int_to_string(question_Number, i) for i in answer_list if i.isdigit()]
    else:
        if (answer_df["agreed_Answer"].iloc[0].isdigit()):
            return answer_int_to_string(question_Number, int(answer_df["agreed_Answer"].iloc[0]))
        else:
            return "na"

In [55]:
# Checklist example
get_consensus('4b537e0ed21179a29ed28da28057d338e67330ae12123ccceba6724f35bd68a4',
              'a23b77bd-9eae-4050-9b5d-e38ecb126226',
              '45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce1626cf4206e159819c',
              2)

['T1.Q2.A4', 'T1.Q2.A5', 'T1.Q2.A7']

In [61]:
# Ordinal example
get_consensus('4b537e0ed21179a29ed28da28057d338e67330ae12123ccceba6724f35bd68a4',
              'a23b77bd-9eae-4050-9b5d-e38ecb126226',
              '45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce1626cf4206e159819c',
              4)

'T1.Q4.A4'

Function that takes in **article_sha256** (string), **tua_uuid** (string), **schema_sha256** (string), and **question_Number** (int), and returns a **tuple containing the question type and number of answer choices, will help with scoring questions**.

If the question_type and num_answer_choices works as expected, we shouldn't need to hardcode the question schema data anymore.

In [52]:
def get_question_meta(article_sha256, tua_uuid, schema_sha256, question_Number):
    answer_row = iaa.loc[(iaa["article_sha256"] == article_sha256)
                         & (iaa["tua_uuid"] == tua_uuid)
                         & (iaa["schema_sha256"] == schema_sha256)
                         & (iaa["question_Number"] == question_Number)]
    
    return (answer_row["question_type"].iloc[0], answer_row["num_answer_choices"].iloc[0])

In [54]:
get_question_meta('4b537e0ed21179a29ed28da28057d338e67330ae12123ccceba6724f35bd68a4',
              'a23b77bd-9eae-4050-9b5d-e38ecb126226',
              '45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce1626cf4206e159819c',
              2)

('checklist', 9)